In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from transformers import pipeline
import pandas as pd

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
model_name = "bigscience/bloomz-560m"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

## Palying with a sample first

In [24]:
def buildprompt(sentence):
    prompt = "Given the sentence delimited by triple backticks ```{" + sentence + "}``` Would you rate the " \
             "previous review as positive, neutral or negative?"
    return prompt

In [25]:
sentence = "go there . dont miss the chance. the food and service was great. just a bit expensive. ," \
           "my bill , alone was 100 $ ( no alcohol included)"

In [26]:
prompt = buildprompt(sentence)

In [27]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_length=200)
print(result)

[{'generated_text': 'Given the sentence delimited by triple backticks ```{go there . dont miss the chance. the food and service was great. just a bit expensive. ,my bill , alone was 100 $ ( no alcohol included)}``` Would you rate the previous review as positive, neutral or negative? positive'}]


## Lets try for one of the topics resulted from the topic modeling approach

In [32]:
# load the original data
df = pd.read_csv('New_Delhi_reviews.csv')
df.dropna(inplace = True)

In [33]:
# using the index as id. This will help to identify the documents after topic modeling
df['id'] = df.index
df

,rating_review,review_full,id
0,5,"Totally in love with the Auro of the place, re...",0
1,5,I went this bar 8 days regularly with my husba...,1
2,5,We were few friends and was a birthday celebra...,2
3,5,Fatjar Cafe and Market is the perfect place fo...,3
4,5,"Hey Guys, if you are craving for pizza and sea...",4
...,...,...,...
147576,5,Near by airport very calm and cool environment...,147576
147577,5,My favourite place to stay. Great service. Ash...,147577
147578,5,"good food with nice decoration, drinks list al...",147578
147579,4,Near to airport .it is fine property. Staff i...,147579


In [35]:
df_tm = pd.read_csv('documents_all.csv', sep='\t')
# the index correspond to the id (order) of the document in the original dataframe. We can use 
# it to identify the document and restore the ratings
df_tm['id'] = df_tm.index
df_tm

,Unnamed: 0,Document,Topic,Name,Representation,Aspect1,Representative_Docs,Top_n_words,Probability,Representative_document,id
0,0,totally love auro place really beautiful quite...,-1,-1_restaurant_restaurants_dinner_meal,"['restaurant', 'restaurants', 'dinner', 'meal'...","['restaurants', 'dinner', 'buffet', 'delhi', '...",['kheer one iconic indian restaurants located ...,restaurant - restaurants - dinner - meal - lun...,0.827076,False,0
1,1,went bar days regularly husband fully satisfie...,21,21_wonderful service_great service_excellent s...,"['wonderful service', 'great service', 'excell...","['wonderful service', 'service staff', 'good s...",['good ambience excellent service staff helpfu...,wonderful service - great service - excellent ...,0.019931,False,1
2,2,friends birthday celebration food good taste r...,11,11_birthday party_birthday celebration_celebra...,"['birthday party', 'birthday celebration', 'ce...","['birthday celebration', 'anniversary dinner',...",['hi guys occasion celebrate birthday marriage...,birthday party - birthday celebration - celebr...,0.045643,False,2
3,3,fatjar cafe market perfect place casual lunch ...,-1,-1_restaurant_restaurants_dinner_meal,"['restaurant', 'restaurants', 'dinner', 'meal'...","['restaurants', 'dinner', 'buffet', 'delhi', '...",['kheer one iconic indian restaurants located ...,restaurant - restaurants - dinner - meal - lun...,0.516001,False,3
4,4,hey guys craving pizza searching visit cafe ye...,-1,-1_restaurant_restaurants_dinner_meal,"['restaurant', 'restaurants', 'dinner', 'meal'...","['restaurants', 'dinner', 'buffet', 'delhi', '...",['kheer one iconic indian restaurants located ...,restaurant - restaurants - dinner - meal - lun...,0.486350,False,4
...,...,...,...,...,...,...,...,...,...,...,...
147574,147574,near airport calm cool environment nice staff ...,22,22_food staff_food service_staff food_service ...,"['food staff', 'food service', 'staff food', '...","['food staff', 'service food', 'nice staff', '...",['amazing experience food really good staff co...,food staff - food service - staff food - servi...,0.041725,False,147574
147575,147575,favourite place stay great service ashish bhar...,-1,-1_restaurant_restaurants_dinner_meal,"['restaurant', 'restaurants', 'dinner', 'meal'...","['restaurants', 'dinner', 'buffet', 'delhi', '...",['kheer one iconic indian restaurants located ...,restaurant - restaurants - dinner - meal - lun...,0.193009,False,147575
147576,147576,good food nice decoration drinks list alot cho...,-1,-1_restaurant_restaurants_dinner_meal,"['restaurant', 'restaurants', 'dinner', 'meal'...","['restaurants', 'dinner', 'buffet', 'delhi', '...",['kheer one iconic indian restaurants located ...,restaurant - restaurants - dinner - meal - lun...,0.712257,False,147576
147577,147577,near airport fine property staff warm friendly...,22,22_food staff_food service_staff food_service ...,"['food staff', 'food service', 'staff food', '...","['food staff', 'service food', 'nice staff', '...",['amazing experience food really good staff co...,food staff - food service - staff food - servi...,0.053849,False,147577


## Focusing on topic 73

In [37]:
list_of_ids = df_tm[df_tm['Topic'] == 73]['id'].to_list()
print(len(list_of_ids))
df_topic_73 = df[df['id'].isin(list_of_ids)]
df_topic_73

120


,rating_review,review_full,id
5424,3,Well at first we were happy to find this place...,5424
7501,1,We have never witnessed such a TAX SCAM of 28%...,7501
8270,3,"While we normally dont eat meals at HRC, we di...",8270
12653,1,This restaurant has a reputation of charging e...,12653
13309,3,I ate twice at Sevilla whilst staying at Clari...,13309
...,...,...,...
132133,3,"Dark,dingy and a little seedy. Now,personally ...",132133
132712,1,On Saturday 12/11/2016 had ordered delivery!.T...,132712
133392,3,I've been so many times to this little place o...,133392
133394,3,Property situated at main road of Pharagunj op...,133394
